In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, Reshape

In [ ]:
mnist = tf.keras.datasets.mnist
(X, y), (X_test, y_test) = mnist.load_data()
X, X_test = X / 255 * 2 - 1, X_test / 255.0 * 2 - 1

In [3]:
X = X.reshape(-1, 28, 28, 1)

In [4]:
def get_generator (latent_space):
    
    i = Input(shape=(latent_space,))
        
    x = Dense(256,activation=LeakyReLU(alpha=0.2)) (i)
    x = BatchNormalization(momentum=0.7) (x)
    x = Dropout (0.2) (x)
    
    x = Dense(512,activation=LeakyReLU(alpha=0.2)) (x)
    x = BatchNormalization(momentum=0.7) (x)
    x = Dropout (0.2) (x)

    x = Dense(1024, activation='tanh') (x)

    x = Reshape ((32, 32, 1)) (x)
    
    x = Conv2D (16, (3, 3), activation=LeakyReLU(alpha=0.2)) (x)
    x = Conv2D (1, (3, 3), activation='tanh') (x)

    model = Model(i, x)
    
    return model

In [5]:
def get_discriminator (img_shape):
    
    i = Input(shape=img_shape)
    
    x = Conv2D (6, (3, 3), activation=LeakyReLU(alpha=0.2)) (i)
    x = Flatten ()(x)
    x = Dense (128, activation=LeakyReLU(alpha=0.2)) (x)
    x = BatchNormalization(momentum=0.7) (x)
    x = Dropout (0.3)(x)
    x = Dense (1, activation='sigmoid') (x)

    model = Model (i, x)
    
    return model

In [6]:
latent_space = 100
img_shape = (28, 28, 1)
generator = get_generator(latent_space)
discriminator = get_discriminator(img_shape)

In [7]:
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy']
)

In [8]:
def get_gan (latent_space):

    discriminator.trainable = False
    
    inputs = Input (shape=latent_space)
    img = generator(inputs)
    pred_d = discriminator(img)

    model = Model (inputs, pred_d)
    return model

In [9]:
gan = get_gan (latent_space)

gan.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy']
)

In [10]:
batch_size = 32
epochs = 30000
period = 1000

In [11]:
def save_img (epoch):
    noise = np.random.randn(batch_size, 100)
    imgs = generator.predict(noise)

    imgs = 0.5 * imgs + 0.5

    fig, axs = plt.subplots(3, 3)
    idx = 0
    for i in range(3):
        for j in range(3):
            axs[i,j].imshow(imgs[idx].reshape(28, 28), cmap='gray')
            axs[i,j].axis('off')
            idx += 1
    fig.savefig("/kaggle/working/%d.png" % epoch)
    plt.close()

In [12]:
d_loss_list = []
g_loss_list = []

In [ ]:
for epoch in range(epochs):

    noise = np.random.randn(batch_size//2, 100)

    x_false, false = generator.predict(noise), np.zeros(batch_size//2)
    
    idx = np.random.randint(0, X.shape[0], batch_size//2)
    x_true, true = X[idx], np.ones(batch_size//2)

    x_final = np.concatenate((x_false, x_true))
    label = np.concatenate((false, true))
    
    d_loss, d_acc = discriminator.train_on_batch(x_final, label)
    
    noise = np.random.randn(batch_size, 100)
    g_loss, g_acc = gan.train_on_batch(noise, np.ones(batch_size))    
    
    noise = np.random.randn(batch_size, 100)
    g_loss, g_acc = gan.train_on_batch(noise, np.ones(batch_size))
    
    d_loss_list.append (d_loss)
    g_loss_list.append (g_loss)
    print ('d_acc : ', d_acc,'g_acc : ', g_acc, 'd_loss : ', d_loss, 'g_loss : ', g_loss)
    
    if epoch % period == 0:
        save_img (epoch)

In [ ]:
plt.plot(d_loss_list, label='d_loss')
plt.plot(g_loss_list, label='g_loss')
plt.legend()